In [175]:
import mlflow
import mlflow.sklearn
import optuna
import numpy as np
import pandas as pd
import os
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFECV
from sklearn.model_selection import cross_val_score, TimeSeriesSplit, train_test_split
from sklearn.metrics import root_mean_squared_error

from boruta import BorutaPy

import lightgbm as lgb
import warnings

warnings.filterwarnings("ignore", category=FutureWarning, module="sklearn.utils.deprecation")


In [180]:
ABT_path = os.path.join('..', 'data', 'ABTs', 'principal_ABT.csv')
ABT = pd.read_csv(ABT_path, index_col= 0)
ABT.head()

ABT = ABT.astype(float)

In [181]:
X = ABT.drop(columns= ['target'])
y = ABT['target']

print(X.shape, y.shape)

(2738, 24) (2738,)


In [183]:
#Separemos un conjunto de test para la los resultados finales
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.1, shuffle= False)

#Unas verificaciones para garantizar la calidad de los datos
print(f'Tamano train: {X_train.shape, y_train.shape}')
print(f'Tamano test: {X_test.shape, y_test.shape}')
print(f'Fecha inicio train {X_train.index.min()} ' , f'Fecha final train {X_train.index.max()} ')
print(f'Fecha inicio test {X_test.index.min()} ' , f'Fecha final test {X_test.index.max()} ')

Tamano train: ((2464, 24), (2464,))
Tamano test: ((274, 24), (274,))
Fecha inicio train 2014-04-17  Fecha final train 2024-01-31 
Fecha inicio test 2024-02-01  Fecha final test 2025-03-06 


In [166]:
def objective_lgb(trial, X, y):
    # Definir hiperparámetros de LightGBM a optimizar, con rangos pensados para reducir sobreajuste
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 25, 30),
        'learning_rate': trial.suggest_float('learning_rate', 1e-8, 1e-5, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 10, 20),
        'max_depth': trial.suggest_int('max_depth', 3, 5),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 50),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.9, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.9, 1.0),
        'random_state': 42,
        'verbosity': -1,
    }
    
    # Utilizamos TimeSeriesSplit con 3 splits
    tscv = TimeSeriesSplit(n_splits=3)
    
    train_rmse_list = []
    val_rmse_list = []
    
    # Para cada fold, aplicamos Boruta y entrenamos el modelo LightGBM en las features seleccionadas
    for train_index, val_index in tscv.split(X):
        X_train_fold = X.iloc[train_index]
        y_train_fold = y.iloc[train_index]
        X_val_fold = X.iloc[val_index]
        y_val_fold = y.iloc[val_index]
        
        # Instanciar un estimador base para Boruta
        base_estimator = lgb.LGBMRegressor(**param, eval_set= [(X_val_fold, y_val_fold)], eval_metric='rmse')
        boruta_selector = BorutaPy(estimator=base_estimator, n_estimators='auto', verbose=0, random_state=42)
        
        # BorutaPy requiere arrays numpy
        boruta_selector.fit(X_train_fold.values, y_train_fold.values)
        
        # Transformar los conjuntos usando la máscara de features seleccionadas
        X_train_sel = boruta_selector.transform(X_train_fold.values)
        X_val_sel = boruta_selector.transform(X_val_fold.values)
        
        # Si Boruta selecciona menos de 1 feature, usamos todas (para evitar errores)
        if X_train_sel.shape[1] < 1:
            X_train_sel = X_train_fold.values
            X_val_sel = X_val_fold.values
        
        # Entrenar el modelo final en el fold con las features seleccionadas
        final_model = lgb.LGBMRegressor(**param)
        final_model.fit(X_train_sel, y_train_fold.values)
        
        # Calcular métricas de entrenamiento y validación
        y_train_pred = final_model.predict(X_train_sel)
        y_val_pred = final_model.predict(X_val_sel)
        
        rmse_train = root_mean_squared_error(y_train_fold, y_train_pred)
        rmse_val = root_mean_squared_error(y_val_fold, y_val_pred)
        
        train_rmse_list.append(rmse_train)
        val_rmse_list.append(rmse_val)
    
    # Promediar las métricas en los folds
    avg_train_rmse = np.mean(train_rmse_list)
    avg_val_rmse = np.mean(val_rmse_list)
    
    # Imprimir las métricas para seguimiento
    print(f"Avg Train RMSE: {avg_train_rmse:.4f} | Avg Validation RMSE: {avg_val_rmse:.4f}")
    
    # Registrar el experimento en MLflow (cada trial se registra como un run anidado)
    with mlflow.start_run(nested=True):
        mlflow.log_params(param)
        mlflow.log_metric("avg_train_rmse", avg_train_rmse)
        mlflow.log_metric("avg_val_rmse", avg_val_rmse)
        # No se loguea el modelo aquí ya que se entrena en cada fold; 
        # el objetivo es optimizar la métrica de validación.
    
    # Optuna busca minimizar el error de validación
    return avg_val_rmse

In [167]:
study = optuna.create_study(direction="minimize", sampler= optuna.samplers.TPESampler(seed= 42))
study.optimize(lambda trial: objective_lgb(trial, X_train, y_train), n_trials= 50)

[I 2025-03-12 23:00:16,788] A new study created in memory with name: no-name-010db96b-5da4-49fb-9d71-c27a4e72bd3b
[I 2025-03-12 23:00:34,048] Trial 0 finished with value: 0.8540299501209924 and parameters: {'n_estimators': 27, 'learning_rate': 7.114476009343412e-06, 'num_leaves': 18, 'max_depth': 4, 'min_child_samples': 16, 'subsample': 0.5779972601681014, 'colsample_bytree': 0.5290418060840998, 'reg_alpha': 0.9866176145774935, 'reg_lambda': 0.9601115011743209}. Best is trial 0 with value: 0.8540299501209924.


Avg Train RMSE: 0.5010 | Avg Validation RMSE: 0.8540


[I 2025-03-12 23:00:45,447] Trial 1 finished with value: 0.8541128962786567 and parameters: {'n_estimators': 29, 'learning_rate': 1.1527987128232397e-08, 'num_leaves': 20, 'max_depth': 5, 'min_child_samples': 18, 'subsample': 0.5909124836035503, 'colsample_bytree': 0.5917022549267169, 'reg_alpha': 0.9304242242959538, 'reg_lambda': 0.9524756431632238}. Best is trial 0 with value: 0.8540299501209924.


Avg Train RMSE: 0.5011 | Avg Validation RMSE: 0.8541


[I 2025-03-12 23:00:50,103] Trial 2 finished with value: 0.8541121302256922 and parameters: {'n_estimators': 27, 'learning_rate': 7.476312062252278e-08, 'num_leaves': 16, 'max_depth': 3, 'min_child_samples': 21, 'subsample': 0.6831809216468459, 'colsample_bytree': 0.728034992108518, 'reg_alpha': 0.9785175961393013, 'reg_lambda': 0.919967378215836}. Best is trial 0 with value: 0.8540299501209924.


...

Avg Train RMSE: 0.5011 | Avg Validation RMSE: 0.8541


[I 2025-03-12 23:10:47,185] Trial 48 finished with value: 0.8540823680597492 and parameters: {'n_estimators': 27, 'learning_rate': 2.5402937505487923e-06, 'num_leaves': 10, 'max_depth': 3, 'min_child_samples': 25, 'subsample': 0.758307986801766, 'colsample_bytree': 0.8012516648698133, 'reg_alpha': 0.9248510535726246, 'reg_lambda': 0.9113062829253293}. Best is trial 41 with value: 0.8539932070634068.


Avg Train RMSE: 0.5011 | Avg Validation RMSE: 0.8541


[I 2025-03-12 23:11:07,856] Trial 49 finished with value: 0.8540295333344282 and parameters: {'n_estimators': 26, 'learning_rate': 7.054763710151258e-06, 'num_leaves': 12, 'max_depth': 4, 'min_child_samples': 18, 'subsample': 0.61544324995944, 'colsample_bytree': 0.6845807675385147, 'reg_alpha': 0.9534261738365837, 'reg_lambda': 0.9320707152444673}. Best is trial 41 with value: 0.8539932070634068.


Avg Train RMSE: 0.5010 | Avg Validation RMSE: 0.8540


In [168]:
best_params =  {'n_estimators': 26, 'learning_rate': 9.89517587604896e-06, 'num_leaves': 10, 'max_depth': 3, 'min_child_samples': 12, 'subsample': 0.761636885611244, 'colsample_bytree': 0.7280669024814017, 'reg_alpha': 0.9334249209243208, 'reg_lambda': 0.9788958090626914}

base_estimator = lgb.LGBMRegressor(**best_params)
boruta_selector = BorutaPy(estimator=base_estimator, n_estimators='auto', verbose=0, random_state=42)

boruta_selector.fit(X_train.values, y_train.values)
X_train_selected = boruta_selector.transform(X_train.values)
X_test_selected = boruta_selector.transform(X_test.values)

best_model = lgb.LGBMRegressor(**best_params)
best_model.fit(X_train_selected, y_train)

y_pred_train = best_model.predict(X_train_selected)
train_rmse = root_mean_squared_error(np.expm1(y_train), np.expm1(y_pred_train))
print("Train RMSE:", train_rmse)

y_pred = best_model.predict(X_test_selected)
test_rmse = root_mean_squared_error(np.expm1(y_test), np.expm1(y_pred))
print("Test RMSE:", test_rmse)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000643 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11256
[LightGBM] [Info] Number of data points in the train set: 1172, number of used features: 48
[LightGBM] [Info] Start training from score 1.693565
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,